# **Chapter 8: Avoiding Hallucinations**

---
**Lesson:**

Some bad news: Claude sometimes "hallucinates" and makes claims that are untrue or unjustified. The good news: there are techniques you can use to minimize hallucinations.				
Below, we'll go over a few of these techniques, namely:

- Giving Claude the option to say it doesn't know the answer to a question 
- Asking Claude to find the evidence before answering 
---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

#Set up model inference modifiers, and ensure Claude v2 is being called from Bedrock 
from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

# **Examples:**

**Example 8.1 - Hungry Hippos**

Here is a question about general factual knowledge to which Claude hallucinates an answer because it's trying to be as helpful as possible.

In [ ]:
response = textgen_llm("""

Human: Who is the heaviest hippo ever recorded?  

Assistant:""")

print_ww(response)

**Example 8.2 - Hungry Hippos 2**

A solution we can try here is to "give Claude an out" — tell Claude that it's OK for it to decline to answer, or to only answer if it actually knows the answer with certainty.

In [ ]:
response = textgen_llm("""

Human: Who is the heaviest hippo ever recorded? Only answer if you know the answer with certainty  

Assistant:""")

print_ww(response)

# **Exercise**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 8.1 - NBA Hallucination**

Adapt the prompt in the cell below to fix Claude's hallucination issue by giving Claude an out. (Kawhi Leonard has only won two NBA championships.) 

In [ ]:
response = textgen_llm("""

Human: In what year did Kawhi Leonard win his third NBA championship?

Assistant:""")

print_ww(response)

# Chapter 8 - END.